In [63]:
# Importing stock ml libraries
import os
import time
import numpy as np
import pandas as pd
from sklearn import metrics
import transformers
import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertModel, BertConfig, BertForSequenceClassification
from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification

In [64]:
# if you are running this notebook from 'home/dev/enefit/notebook'. 
os.chdir('..') # else adjust to point to the root of the project.

In [65]:
%%capture output

%load_ext kedro.ipython
%reload_kedro

if 'output' in locals() and 'error' in output.stderr:
    output.show()

In [96]:
# Setting up the device for GPU usage
from torch import cuda
device = torch.device('cuda:0' if cuda.is_available() else 'cpu:0')
device

device(type='cuda', index=0)

In [67]:
# Root label (source = ASRS coding forms) : order = by descending frequency
anomaly_labels=['Deviation / Discrepancy - Procedural',
                    'Aircraft Equipment',
                    'Conflict',
                    'Inflight Event / Encounter',
                    'ATC Issue',
                    'Deviation - Altitude',
                    'Deviation - Track / Heading',
                    'Ground Event / Encounter',
                    'Flight Deck / Cabin / Aircraft Event',
                    'Ground Incursion',
                    'Airspace Violation',
                    'Deviation - Speed',
                    'Ground Excursion',
                    'No Specific Anomaly Occurred']

In [68]:
# Function to check prefixes and include 'Other' category
def check_prefixes(anomaly, prefixes):
    if pd.isna(anomaly):
        # Return a series of 0s if the anomaly is NaN
        return pd.Series({prefix: 0 for prefix in prefixes + ['Other']})
    
    split_anomalies = [item.strip() for item in anomaly.split(';')]
    prefix_matches = {prefix: any(item.startswith(prefix) for item in split_anomalies) for prefix in prefixes}
    prefix_matches['Other'] = not any(prefix_matches.values())  # If no prefix matches, this is 'Other'
    return pd.Series(prefix_matches)

In [69]:
# from google.colab import drive
# drive.mount('/content/drive')

drop the NaN values in Anomaly?

In [70]:
#loaded_data = pd.read_pickle("./data/train_data_final.pkl")
#pd.to_pickle(loaded_data, "./data/train_data_final.pkl")

In [71]:
train_df = catalog.load('train_data_int')
test_df = catalog.load('test_data_int')

                    INFO     Loading data from 'train_data_int' (ParquetDataset)...             ]8;id=607741;file:///home/ensai/.pyenv/versions/3.10.12/envs/safran-nlp/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=694566;file:///home/ensai/.pyenv/versions/3.10.12/envs/safran-nlp/lib/python3.10/site-packages/kedro/io/data_catalog.py#502\502]8;;\

[12/06/23 06:45:51] INFO     Loading data from 'test_data_int' (ParquetDataset)...              ]8;id=269041;file:///home/ensai/.pyenv/versions/3.10.12/envs/safran-nlp/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=629297;file:///home/ensai/.pyenv/versions/3.10.12/envs/safran-nlp/lib/python3.10/site-packages/kedro/io/data_catalog.py#502\502]8;;\

In [72]:
# loaded_data = pd.read_pickle("./data/train_data_final.pkl")

#train_df = loaded_data[0]
#print("\nA Dataframe with", len(train_df), "entries has been loaded")

# Apply this function to each row in the 'Anomaly' column
#train_anomaly_encoding = train_df['Anomaly'].apply(lambda x: check_prefixes(x, anomaly_labels))
#train_df['anomaly_encoding'] = train_anomaly_encoding.values.tolist()
#train_df.head()

In [73]:
#loaded_data = pd.read_pickle("./data/test_data_final.pkl")

#test_df = loaded_data[0]
#print("\nA Dataframe with", len(test_df), "entries has been loaded")

# Apply this function to each row in the 'Anomaly' column
#test_anomaly_encoding = test_df['Anomaly'].apply(lambda x: check_prefixes(x, anomaly_labels))
#test_df['anomaly_encoding'] = test_anomaly_encoding.values.tolist()
#test_df.head()


In [74]:

train_anomaly_encoding = train_df['Anomaly'].apply(lambda x: check_prefixes(x, anomaly_labels))
train_df['anomaly_encoding'] = train_anomaly_encoding.values.tolist()

test_anomaly_encoding = test_df['Anomaly'].apply(lambda x: check_prefixes(x, anomaly_labels))
test_df['anomaly_encoding'] = test_anomaly_encoding.values.tolist()

In [75]:
#
# MODEL_NAME = "model"
MODEL_NAME = None
MODEL_DIRECTORY = "model_save"


# Sections of configBertTokenizer
# Defining some key variables that will be used later on in the training
MAX_LEN = 512
TRAIN_BATCH_SIZE = 32 # 32 Size for NASA
VALID_BATCH_SIZE = 128
EPOCHS = 5 # 5 Epochs for NASA
LEARNING_RATE = 1e-05 * 2 # 0.00002 Rate for NASA
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# tokenizer = AutoTokenizer.from_pretrained("NASA-AIML/MIKA_SafeAeroBERT")

In [76]:
class CustomDataset(Dataset):

    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.narrative = dataframe.Narrative
        self.targets = self.data.anomaly_encoding
        self.max_len = max_len

    def __len__(self):
        return len(self.narrative)

    def __getitem__(self, index):
        narrative = str(self.narrative.iloc[index])
        narrative = " ".join(narrative.split())

        inputs = self.tokenizer(
            narrative,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]


        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.targets.iloc[index], dtype=torch.float)
        }


In [77]:
# Creating the dataset and dataloader for the neural network
print("TRAIN Dataset: {}".format(train_df.shape))
print("TEST Dataset: {}".format(test_df.shape))

training_set = CustomDataset(train_df, tokenizer, MAX_LEN)
testing_set = CustomDataset(test_df, tokenizer, MAX_LEN)

TRAIN Dataset: (97417, 97)
TEST Dataset: (10824, 97)


In [78]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 2
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 2
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

In [79]:
# Compute weights for loss function
pos_num = torch.zeros(15)
for _, data in enumerate(training_loader, 0) :
    targets = data['targets']
    pos_num += torch.sum(targets, axis=0)
nobs = len(training_loader.dataset)
pos_weight = (nobs - pos_num) / pos_num

In [80]:
torch.cuda.empty_cache()

In [97]:
class BERTClass(torch.nn.Module):
    def __init__(self, num_labels=15):
        super(BERTClass, self).__init__()
        self.l1 = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=num_labels,)

    def forward(self, ids, mask, token_type_ids):
        output = self.l1(ids, attention_mask=mask, token_type_ids=token_type_ids)
        return output.logits

model = BERTClass()

# Freeze all layers in the model
for param in model.parameters():
    param.requires_grad = False

# Unfreeze the classifier and pooler layers
for param in model.l1.classifier.parameters():
    param.requires_grad = True

for param in model.l1.bert.pooler.parameters():
    param.requires_grad = True

model.to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:23                                                                                   │
│                                                                                                  │
│   20 for param in model.l1.bert.pooler.parameters():                                             │
│   21 │   param.requires_grad = True                                                              │
│   22                                                                                             │
│ ❱ 23 model.to(device)                                                                            │
│   24                                                                                             │
│                                                                                                  │
│ /home/ensai/.pyenv/versions/3.10.12/envs/safran-nlp/lib/python3.10/site-packages/torch/nn/module │
│ s/module.py:1160 in to                                                                           │
│                                                                                                  │
│   1157 │   │   │   │   │   │   │   non_blocking, memory_format=convert_to_format)                │
│   1158 │   │   │   return t.to(device, dtype if t.is_floating_point() or t.is_complex() else No  │
│   1159 │   │                                                                                     │
│ ❱ 1160 │   │   return self._apply(convert)                                                       │
│   1161 │                                                                                         │
│   1162 │   def register_full_backward_pre_hook(                                                  │
│   1163 │   │   self,                                                                             │
│                                                                                                  │
│ /home/ensai/.pyenv/versions/3.10.12/envs/safran-nlp/lib/python3.10/site-packages/torch/nn/module │
│ s/module.py:810 in _apply                                                                        │
│                                                                                                  │
│    807 │   def _apply(self, fn, recurse=True):                                                   │
│    808 │   │   if recurse:                                                                       │
│    809 │   │   │   for module in self.children():                                                │
│ ❱  810 │   │   │   │   module._apply(fn)                                                         │
│    811 │   │                                                                                     │
│    812 │   │   def compute_should_use_set_data(tensor, tensor_applied):                          │
│    813 │   │   │   if torch._has_compatible_shallow_copy_type(tensor, tensor_applied):           │
│                                                                                                  │
│ /home/ensai/.pyenv/versions/3.10.12/envs/safran-nlp/lib/python3.10/site-packages/torch/nn/module │
│ s/module.py:810 in _apply                                                                        │
│                                                                                                  │
│    807 │   def _apply(self, fn, recurse=True):                                                   │
│    808 │   │   if recurse:                                                                       │
│    809 │   │   │   for module in self.children():                                                │
│ ❱  810 │   │   │   │   module._apply(fn)                                                         │
│    811 │   │                                                                                     │
│    812 │   │   def compute_should_use_set_data(tensor, tens

In [82]:
class SafeAeroBERTClass(torch.nn.Module):
    def __init__(self, num_labels=15):
        super(SafeAeroBERTClass, self).__init__()
        self.l1 = AutoModelForSequenceClassification.from_pretrained("NASA-AIML/MIKA_SafeAeroBERT", num_labels=num_labels,)

    def forward(self, ids, mask, token_type_ids):
        output = self.l1(ids, attention_mask=mask, token_type_ids=token_type_ids)
        return output.logits

model = SafeAeroBERTClass()

# Freeze all layers in the model
for param in model.parameters():
    param.requires_grad = False

# Unfreeze the classifier and pooler layers
for param in model.l1.classifier.parameters():
    param.requires_grad = True

for param in model.l1.bert.pooler.parameters():
    param.requires_grad = True

model.to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at NASA-AIML/MIKA_SafeAeroBERT and are newly initialized: ['bert.pooler.dense.bias', 'classifier.bias', 'bert.pooler.dense.weight', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:23                                                                                   │
│                                                                                                  │
│   20 for param in model.l1.bert.pooler.parameters():                                             │
│   21 │   param.requires_grad = True                                                              │
│   22                                                                                             │
│ ❱ 23 model.to(device)                                                                            │
│   24                                                                                             │
│                                                                                                  │
│ /home/ensai/.pyenv/versions/3.10.12/envs/safran-nlp/lib/python3.10/site-packages/torch/nn/module │
│ s/module.py:1160 in to                                                                           │
│                                                                                                  │
│   1157 │   │   │   │   │   │   │   non_blocking, memory_format=convert_to_format)                │
│   1158 │   │   │   return t.to(device, dtype if t.is_floating_point() or t.is_complex() else No  │
│   1159 │   │                                                                                     │
│ ❱ 1160 │   │   return self._apply(convert)                                                       │
│   1161 │                                                                                         │
│   1162 │   def register_full_backward_pre_hook(                                                  │
│   1163 │   │   self,                                                                             │
│                                                                                                  │
│ /home/ensai/.pyenv/versions/3.10.12/envs/safran-nlp/lib/python3.10/site-packages/torch/nn/module │
│ s/module.py:810 in _apply                                                                        │
│                                                                                                  │
│    807 │   def _apply(self, fn, recurse=True):                                                   │
│    808 │   │   if recurse:                                                                       │
│    809 │   │   │   for module in self.children():                                                │
│ ❱  810 │   │   │   │   module._apply(fn)                                                         │
│    811 │   │                                                                                     │
│    812 │   │   def compute_should_use_set_data(tensor, tensor_applied):                          │
│    813 │   │   │   if torch._has_compatible_shallow_copy_type(tensor, tensor_applied):           │
│                                                                                                  │
│ /home/ensai/.pyenv/versions/3.10.12/envs/safran-nlp/lib/python3.10/site-packages/torch/nn/module │
│ s/module.py:810 in _apply                                                                        │
│                                                                                                  │
│    807 │   def _apply(self, fn, recurse=True):                                                   │
│    808 │   │   if recurse:                                                                       │
│    809 │   │   │   for module in self.children():                                                │
│ ❱  810 │   │   │   │   module._apply(fn)                                                         │
│    811 │   │                                                                                     │
│    812 │   │   def compute_should_use_set_data(tensor, tens

In [83]:
loss_fn = torch.nn.BCEWithLogitsLoss(pos_weight=pos_weight)  # compute weighted loss for unbalanced dataset
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)
metrics_dict = {
    "Accuracy": metrics.accuracy_score,
    "F1 Micro Score": lambda y_true, y_pred: metrics.f1_score(y_true, y_pred, average='micro', zero_division=1),
    "F1 Macro Score": lambda y_true, y_pred: metrics.f1_score(y_true, y_pred, average='macro', zero_division=1),
    "F1 Scores per Class": lambda y_true, y_pred: metrics.f1_score(y_true, y_pred, average=None, zero_division=1)
}


In [84]:
def save_model(model, epoch, directory='model_save', model_name=None):
    """
    Saves the model state.

    Args:
    model (torch.nn.Module): The model to save.
    epoch (int): The current epoch number.
    file_path (str): Base directory to save the models.
    """
    if model_name is None:
        model_name = model.__class__.__name__

    if not os.path.exists(directory):
        os.makedirs(directory)
    
    file_path = os.path.join(directory, f"{model_name}_epoch_{epoch}.pth")

    torch.save(model.state_dict(), file_path)
    print(f'Model saved at {file_path}')


In [85]:
def find_last_saved_epoch(directory='model_save', model_name=None):
    """
    Finds the last saved epoch number in the specified directory.

    Args:
    file_path (str): The directory where models are saved.

    Returns:
    int: The last saved epoch number. Returns -1 if no saved model is found.
    """
    if model_name is None:
        model_name = model.__class__.__name__

    # Check if the directory exists, and create it if it doesn't
    if not os.path.exists(directory):
        return -1

    saved_epochs = []
    for filename in os.listdir(directory):
        if model_name is None or filename.startswith(model_name):
            parts = filename.replace('.pth', '').split('_')
            if parts[-2] == 'epoch':
                try:
                    saved_epochs.append(int(parts[-1]))
                except ValueError:
                    pass
    
    return max(saved_epochs, default=-1)

In [86]:
def load_model(model, directory='model_save', model_name=None, epoch=None):
    """
    Loads the model state.

    Args:
    model (torch.nn.Module): The model to load state into.
    file_path (str): Path to the saved model file.
    """
    if model_name is None:
        model_name = model.__class__.__name__

    if epoch is None:
        epoch = find_last_saved_epoch(directory, model_name)
        if epoch == -1:
            print("No saved model found.")
            return
    
    file_path = os.path.join(directory, f"{model_name}_epoch_{epoch}.pth")
    if not os.path.exists(file_path):
        print(f"No model file found at {file_path}")
        return

    model.load_state_dict(torch.load(file_path))
    model.to(device)
    print(f'Model loaded from {file_path}')

In [87]:
def process_batch(model, batch_data, device, loss_fn, mode, optimizer=None):
    ids = batch_data['ids'].to(device, dtype=torch.long)
    mask = batch_data['mask'].to(device, dtype=torch.long)
    token_type_ids = batch_data['token_type_ids'].to(device, dtype=torch.long)
    targets = batch_data['targets'].to(device, dtype=torch.float)

    if mode == 'train':
        optimizer.zero_grad()
        outputs = model(ids, mask, token_type_ids)
        loss = loss_fn(outputs, targets)
        loss.backward()
        optimizer.step()
    else:
        with torch.no_grad():
            outputs = model(ids, mask, token_type_ids)
            loss = loss_fn(outputs, targets)

    return outputs, targets, loss


In [88]:
def calculate_metrics(metrics_dict, targets, outputs):
    results = {}
    labels = anomaly_labels + ["Other"]
    for metric_name, metric_fn in metrics_dict.items():
        if metric_name == "F1 Scores per Class":
            # Calculate F1 score for each class and associate with labels
            f1_scores = metric_fn(targets, outputs)
            for i, score in enumerate(f1_scores):
                label = labels[i] if i < len(labels) else f"Class {i}"
                results[f"F1 Score - {label}"] = score
        else:
            results[metric_name] = metric_fn(targets, outputs)
    return results

In [89]:
def print_metrics_results(metrics_results):
    labels = anomaly_labels + ["Other"]
    for metric, value in metrics_results.items():
        if isinstance(value, np.ndarray):
            # Handling per-class metrics
            for i, score in enumerate(value):
                label = labels[i] if i < len(labels) else f"Class {i}"
                print(f"{metric} ({label}): {score:.4f}")
        else:
            # Handling overall metrics
            print(f"{metric}: {value:.4f}")

In [90]:
def print_batch_results(mode, epoch, batch, dataset_size, loss, batch_metrics_results, start_time, batch_start_time, batch_size):
    current_time = time.time()
    elapsed_time = current_time - start_time
    batch_time_ms = (current_time - batch_start_time) * 1000

    current = (batch + 1) * batch_size
    metric_str = ", ".join([f"{metric}: {value:.4f}" for metric, value in batch_metrics_results.items()])
    epoch_str = f"Epoch: {epoch+1}, " if epoch is not None else ""
    
    print(f"\r{mode.capitalize()} - {epoch_str}Batch: {batch+1} [{current:>5d}/{dataset_size:>5d}], "
          f"Time: {elapsed_time:.0f}s {batch_time_ms:.0f}ms/step, Loss: {loss:>7f}, {metric_str}", end="")


In [91]:
def process_batches(mode, model, loader, device, loss_fn, metrics_dict, optimizer=None, epoch=None):
    model.train() if mode == 'train' else model.eval()
    total_loss = 0.0
    all_targets = []
    all_outputs = []
    start_time = time.time()

    for batch, data in enumerate(loader, 0):
        batch_start_time = time.time()
        
        outputs, targets, loss = process_batch(model, data, device, loss_fn, mode, optimizer)
        total_loss += loss.item()

        outputs_binary = torch.sigmoid(outputs).cpu().detach().numpy() >= 0.5
        targets = targets.cpu().detach().numpy()
        all_outputs.extend(outputs_binary)
        all_targets.extend(targets)

        batch_metrics_results = calculate_metrics(metrics_dict, targets, outputs_binary)
        batch_size = targets.shape[0]
        print_batch_results(mode, epoch, batch, len(loader.dataset), loss.item(), batch_metrics_results, start_time, batch_start_time, batch_size)

    print()
    avg_loss = total_loss / len(loader)
    return avg_loss, all_outputs, all_targets

In [92]:
def evaluate(model, validation_loader, loss_fn, metrics_dict, device):
    avg_val_loss, val_outputs, val_targets = process_batches('evaluate', model, validation_loader, device, loss_fn, metrics_dict)
    val_outputs = np.array(val_outputs) >= 0.5

    metrics_results = {metric_name: metric_fn(val_targets, val_outputs) for metric_name, metric_fn in metrics_dict.items()}

    print(f"Evaluation Results:")
    print(f"Average Loss: {avg_val_loss:.4f}")
    print_metrics_results(metrics_results)

    return avg_val_loss, metrics_results


In [93]:
def train(model, epoch, training_loader, validation_loader, optimizer, loss_fn, metrics_dict, device):
    print(f"Training Epoch {epoch + 1}")

    # Training phase
    avg_train_loss, _, _ = process_batches('train', model, training_loader, device, loss_fn, metrics_dict, optimizer, epoch=epoch)
    print(f"Average Training Loss for Epoch {epoch + 1}: {avg_train_loss:.4f}")

    # Validation phase
    if validation_loader is not None:
        avg_val_loss, val_metrics_results = evaluate(model, validation_loader, loss_fn, metrics_dict, device)
    else:
        avg_val_loss = None
        val_metrics_results = {}

    return avg_train_loss, avg_val_loss, val_metrics_results


In [94]:
last_saved_epoch = find_last_saved_epoch(directory=MODEL_DIRECTORY, model_name=MODEL_NAME)

start_epoch = last_saved_epoch + 1 if last_saved_epoch != -1 else 0
if last_saved_epoch != -1:
    load_model(model, directory=MODEL_DIRECTORY, model_name=MODEL_NAME, epoch=last_saved_epoch)
    print(f"Loaded model training from epoch {start_epoch}")
else:
    print("No saved model found.")

if start_epoch < EPOCHS:
    print(f"Resuming training from epoch {start_epoch + 1}")

for epoch in range(start_epoch, EPOCHS):
    train_loss, val_loss, val_metrics = train(model, epoch, training_loader, testing_loader, optimizer, loss_fn, metrics_dict, device)
    save_model(model, epoch, directory=MODEL_DIRECTORY, model_name=MODEL_NAME)
    # Additional epoch-level processing if needed

# Testing phase
avg_test_loss, test_metrics_results = evaluate(model, testing_loader, loss_fn, metrics_dict, device)
print(f"Test Results:")
print(f"Average Loss: {avg_test_loss}")
print_metrics_results(test_metrics_results)


No saved model found.
Resuming training from epoch 1
Training Epoch 1


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:14                                                                                   │
│                                                                                                  │
│   11 │   print(f"Resuming training from epoch {start_epoch + 1}")                                │
│   12                                                                                             │
│   13 for epoch in range(start_epoch, EPOCHS):                                                    │
│ ❱ 14 │   train_loss, val_loss, val_metrics = train(model, epoch, training_loader, testing_loa    │
│   15 │   save_model(model, epoch, directory=MODEL_DIRECTORY, model_name=MODEL_NAME)              │
│   16 │   # Additional epoch-level processing if needed                                           │
│   17                                                                                             │
│                                                                                                  │
│ in train:5                                                                                       │
│                                                                                                  │
│    2 │   print(f"Training Epoch {epoch + 1}")                                                    │
│    3 │                                                                                           │
│    4 │   # Training phase                                                                        │
│ ❱  5 │   avg_train_loss, _, _ = process_batches('train', model, training_loader, device, loss    │
│    6 │   print(f"Average Training Loss for Epoch {epoch + 1}: {avg_train_loss:.4f}")             │
│    7 │                                                                                           │
│    8 │   # Validation phase                                                                      │
│                                                                                                  │
│ in process_batches:11                                                                            │
│                                                                                                  │
│    8 │   for batch, data in enumerate(loader, 0):                                                │
│    9 │   │   batch_start_time = time.time()                                                      │
│   10 │   │                                                                                       │
│ ❱ 11 │   │   outputs, targets, loss = process_batch(model, data, device, loss_fn, mode, optim    │
│   12 │   │   total_loss += loss.item()                                                           │
│   13 │   │                                                                                       │
│   14 │   │   outputs_binary = torch.sigmoid(outputs).cpu().detach().numpy() >= 0.5               │
│                                                                                                  │
│ in process_batch:2                                                                               │
│                                                                                                  │
│    1 def process_batch(model, batch_data, device, loss_fn, mode, optimizer=None):                │
│ ❱  2 │   ids = batch_data['ids'].to(device, dtype=torch.long)                                    │
│    3 │   mask = batch_data['mask'].to(device, dtype=torch.long)                                  │
│    4 │   token_type_ids = batch_data['token_type_ids'].to(device, dtype=torch.long)              │
│    5 │   targets = batch_data['targets'].to(device, dtype=torch.float)                           │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
RuntimeError: CUDA error: unspecified launch failure
CUDA ker

In [28]:
# from torch.nn.utils.rnn import pad_sequence
# def collate_fn(batch):
#     ids = [item['ids'].clone().detach() for item in batch]
#     masks = [item['mask'].clone().detach() for item in batch]
#     token_type_ids = [item['token_type_ids'].clone().detach() for item in batch]
#     targets = [item['targets'].clone().detach() for item in batch]

#     # Padding the sequences to the maximum length in this batch
#     ids = pad_sequence(ids, batch_first=True, padding_value=tokenizer.pad_token_id)
#     masks = pad_sequence(masks, batch_first=True, padding_value=0)
#     token_type_ids = pad_sequence(token_type_ids, batch_first=True, padding_value=0)

#     targets = torch.stack(targets)

#     return {
#         'ids': ids,
#         'mask': masks,
#         'token_type_ids': token_type_ids,
#         'targets': targets
#     }